In [ ]:
# hide
# default_exp core
# all_slow

# fastcgi API

> API details for fastcgi

In [ ]:
#export
from fastcore.foundation import *
from fastcore.utils import *
from fastcore.meta import *

import struct,socketserver
from enum import Enum
from collections import defaultdict

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *
import subprocess

This library follows the [FastCGI spec](http://www.mit.edu/~yandros/doc/specs/fcgi-spec.html). It only supports the *Responder* role, and does not support multiplexing (which is not supported by any servers, so is unlikely to be an issue).

## Helpers

You probably won't need to use these helpers directly; they are made public and documented in case you want to customize anything.

In [ ]:
#export
Request = Enum('Request', 'BEGIN_REQUEST ABORT_REQUEST END_REQUEST PARAMS STDIN '
               'STDOUT STDERR DATA GET_VALUES GET_VALUES_RESULT')
Role = Enum('Role', 'RESPONDER AUTHORIZER FILTER')
Status = Enum('Status', 'REQUEST_COMPLETE CANT_MPX_CONN OVERLOADED UNKNOWN_ROLE')

These `enum`s are used throughout the library, and have the same meanings as the `FCGI_` constants `#define`d in the spec.

In [ ]:
for o in Request,Role,Status: print(list(o))

[<Request.BEGIN_REQUEST: 1>, <Request.ABORT_REQUEST: 2>, <Request.END_REQUEST: 3>, <Request.PARAMS: 4>, <Request.STDIN: 5>, <Request.STDOUT: 6>, <Request.STDERR: 7>, <Request.DATA: 8>, <Request.GET_VALUES: 9>, <Request.GET_VALUES_RESULT: 10>]
[<Role.RESPONDER: 1>, <Role.AUTHORIZER: 2>, <Role.FILTER: 3>]
[<Status.REQUEST_COMPLETE: 1>, <Status.CANT_MPX_CONN: 2>, <Status.OVERLOADED: 3>, <Status.UNKNOWN_ROLE: 4>]


In [ ]:
#export
def unpack_from(fmt, s, offset=0, prefix="!"):
    "`struct.unpack_from` with `prefix`"
    return struct.unpack_from(prefix+fmt, s, offset)

In [ ]:
#export
def pack(s, *args, prefix="!"):
    "`struct.pack` with `prefix`"
    return struct.pack(prefix+s, *args)

Since fastcgi uses "network order" in its binary protocol, we define pack and unpack functions that use that default.

In [ ]:
#export
_REC_STRUCT = 'BBHHbb'
_chk_typs = Request.STDIN,Request.DATA

In [ ]:
#export
def pack_rec(typ, c=b''):
    "Create a fastcgi binary record containing optional content `c`"
    if isinstance(typ,Request): typ=typ.value
    return pack(_REC_STRUCT, 1, typ, 1, len(c), 0, 0) + c

In [ ]:
#export
def unpack_sz(fmt, s, offset=0, prefix="!"):
    "`unpack_from`, returning new `offset` based on size of `fmt`"
    sz = struct.calcsize(fmt)
    res = unpack_from(fmt, s, offset, prefix)
    return offset+sz,(res[0] if len(res)==1 else res)

In [ ]:
#export
def unpack_rec(c,i):
    "Unpack a fastcgi binary record starting at offset `i`"
    i,(_,typ,_,contentlen,padlen,_) = unpack_sz(_REC_STRUCT, c, i)
    i,content = unpack_sz(f'{contentlen}s{"x"*padlen}', c, i)
    return i,(Request(typ),content)

In [ ]:
#export
def unpack_content(typ, c):
    "Unpack the content section of a fastcgi binary record of `typ`"
    if typ==Request.BEGIN_REQUEST: return unpack_from('Hb5s', c)[:2]
    if typ==Request.ABORT_REQUEST: return ()
    raise Exception(typ)

In [ ]:
#export
def _loop_thru(data,f,i=0):
    "Loop thru `data`, calling `f`, which updates `i`"
    while i<len(data):
        i,res = f(data,i)
        yield res

def _param_sz(c,i):
    # See http://www.mit.edu/~yandros/doc/specs/fcgi-spec.html#S3.4
    i,l = unpack_sz('b', c, i)
    return unpack_sz('L', c, i-1) & 0x7fffff if l>>7 else i,l

def _param(c, i):
    # See http://www.mit.edu/~yandros/doc/specs/fcgi-spec.html#S3.4
    i,lk = _param_sz(c,i)
    i,lv = _param_sz(c,i)
    i,res = unpack_sz(f'{lk}s{lv}s', c, i)
    return i,res

def _params(c): return {k.decode():v.decode() for k,v in _loop_thru(c,_param)}
def _records(c): return _loop_thru(c,unpack_rec)

In [ ]:
#export
class _Stream:
    def __init__(self): self.buf,self.done = b'',False
    def __repr__(self): return str(self.buf)
    def __str__(self): return self.buf.decode()

    def append(self,d):
        self.buf += d
        self.done = not d

In [ ]:
#export
class FcgiHandler(socketserver.BaseRequestHandler):
    def setup(self):
        while not self._recv(self.request.recv(68000)): pass

    def finish(self):
        self._end(Request.STDOUT)
        self._end(Request.STDERR)
        self._send(Request.END_REQUEST, pack('LBBBB', 0, Status.REQUEST_COMPLETE.value, 0,0,0))

    def _recv(self,d):
        for typ,c in _records(d):
            if typ in (Request.PARAMS,*_chk_typs): self.streams[typ].append(c)
            else: getattr(self,'_'+typ.name)(*unpack_content(typ,c))
            if typ in _chk_typs: self.sz += len(c)
        if self.sz>=self.length: return True

    def __getitem__(self, k):
        if isinstance(k,str): k = Request[k.upper()]
        return self.streams[k] if k in self.streams else None
    
    def _ABORT_REQUEST(self): self.sz=self.length

    def _BEGIN_REQUEST(self,role,keep_conn):
        self.streams,self.sz = defaultdict(_Stream),0
        assert Role(role)==Role.RESPONDER, f"{role} not supported"
        assert not keep_conn, "FCGI_KEEP_CONN not supported"

    def send(self, c, err=False):
        "Queue content `c` for sending"
        stream = self.streams[Request.STDERR if err else Request.STDOUT]
        stream.append(c if isinstance(c,bytes) else bytes(c, 'utf8'))

    def _send(self, stream, c=b''): self.request.send(pack_rec(stream, c))

    def _end(self, stream):
        if stream not in self.streams: return
        for o in chunked(self[stream].buf, 64000): self._send(stream, bytes(o))
        self._send(stream)

    def __exit__(self, exc_type, exc_value, traceback): self.close()
    def __enter__(self): return self

    @property
    def stdin(self): return str(self['stdin'])
    @property
    def params(self):
        p = self['PARAMS']
        return _params(p.buf) if p else {}
    @property
    def length(self): return int(self.params.get('CONTENT_LENGTH', 1))

This is used in much the same way as [BaseRequestHandler](https://docs.python.org/3/library/socketserver.html#request-handler-objects), except that receiving the data is handled for you before your `handle` method is called. All headers are available in the `params` dictionary, and `stdin` contains the data sent to your handler.

Use `send` to send data to the client, and add `err=True` to pass it as stderr.

Here's an example subclass:

In [ ]:
class TestHandler(FcgiHandler):
    def handle(self):
        print('query:', self.params['QUERY_STRING'])
        print('content type:', self.params['HTTP_CONTENT_TYPE'])
        print('stdin:', self.stdin)
        self.send("Content-type: text/html\r\n\r\n<html>foobar</html>\n")

To test it, we'll use an http->fcgi proxy, we can download `http2fcgi` and run it in the background as follows:

In [ ]:
p = Path('test.sock').absolute()

run('./get_http2fcgi.sh')
proc = subprocess.Popen(f'./http2fcgi -fcgi unix:///{p} -ext py'.split())

We can now test the handler by running a server in the background...

In [ ]:
@threaded
def _f():
    with socketserver.UnixStreamServer(str(p), TestHandler) as server: server.handle_request()

if p.exists(): p.unlink()
t = _f()

...and use `curl` to test it:

In [ ]:
!curl 'http://localhost:6065/setup.py?a=1' -X POST -H "Content-Type: application/json" -d test

query: a=1
content type: application/json
stdin: test
<html>foobar</html>


Finally, we kill the `http2fcgi` background process.

In [ ]:
proc.terminate()

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
